<a href="https://colab.research.google.com/github/ekqlsrla/TextBook/blob/main/HandsOn_MachineLearning/Chapter07_%EC%95%99%EC%83%81%EB%B8%94_%ED%95%99%EC%8A%B5%EA%B3%BC_%EB%9E%9C%EB%8D%A4_%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Chapter 07 앙상블 학습과 랜덤 포레스트**
---
---

## 7.1 투표 기반 분류

1. 직접 투표 분류기 : 다수결 투표로 정해지는 분류

In [1]:
#사이킷런의 투표 기반 분류기를 만들고 훈련시키는 코드

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting = 'hard'
)

voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [2]:
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf) :
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test,y_pred))

#투표 기반 분류기가 다른 개별 분류기보다 성능이 조금 더 높은 것을 확인할 수 있음

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.896


* 모든 분류기가 클래스의 확률을 예측할 수 있으면, 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있음 **= 간접투표**
* `voting = "soft"`로 바꾸고, 모든 분류기가 클래스의 확률을 추정할 수 있으면
됨

---
## 7.2 배깅과 페이스팅

1. **배깅** | 훈련 세트에서 중복을 허용하여 샘플링하는 방식

2. **페이스팅** | 중복을 허용하지 않고 샘플링 하는 방식

* 모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만듦. 수집함수는 전형적으로 분류일 때는 **통계적 최빈값**이고 회귀에 대해서는 **평균**을 계산함

### 1) 사이킷런의 배깅과 페이스팅

* `bootstrap= False`로 지정하면 페이스팅을 사용할 수 있음

In [3]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    max_samples = 100, bootstrap = True, n_jobs = -1
)

bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)

### 2) oob 평가

* 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있음. 예측기가 훈련되는 동안에는 oob 샘플을 사용하지 않으므로 별도의 검증 세트를 사용하지 않고 oob 샘플을 사용해 평가할 수 있음

In [4]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    bootstrap = True, n_jobs = -1, oob_score = True
)

bag_clf.fit(X_train,y_train)
bag_clf.oob_score_

0.8986666666666666

In [5]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.904

* 결정 함수의 값 : 결정함수는 각 훈련 샘플의 클래스 확률을 반환
  * 다음 예를 보면 oob 평가는 첫 번째 훈련 샘플이 양성 클래스에 속할 확률을 65%로 추정

In [6]:
bag_clf.oob_decision_function_

array([[0.44262295, 0.55737705],
       [0.34554974, 0.65445026],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.07692308, 0.92307692],
       [0.33519553, 0.66480447],
       [0.0049505 , 0.9950495 ],
       [0.99444444, 0.00555556],
       [0.97368421, 0.02631579],
       [0.78362573, 0.21637427],
       [0.01197605, 0.98802395],
       [0.75409836, 0.24590164],
       [0.86631016, 0.13368984],
       [0.98514851, 0.01485149],
       [0.06976744, 0.93023256],
       [0.        , 1.        ],
       [0.98019802, 0.01980198],
       [0.95108696, 0.04891304],
       [1.        , 0.        ],
       [0.02673797, 0.97326203],
       [0.31746032, 0.68253968],
       [0.90659341, 0.09340659],
       [1.        , 0.        ],
       [0.97382199, 0.02617801],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.69512195, 0.30487805],
       [0.

---
## 7.3 랜덤 패치와 랜덤 서브스페이스

* `BaggingClassifier`는 특성 샘플링도 지원
  * *max_features,bootstrap_features*
  * **특성**에 대한 샘플링 : 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련

* **랜덤패치방식** | 훈련 특성과 샘플을 모두 샘플링하는 것

* **랜덤서브스페이스방식** | 훈련 샘플을 **모두 사용**`bootstrap = False, max_samples = 1.0`하고 특성은 **샘플링**하는 것 `bootstrap_features = True, max_features < 1.0 `

---

## 7.4 랜덤 포레스트

* 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블
  * *max_samples*를 훈련 세트의 크기로 지정

In [7]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators = 500, max_leaf_nodes = 16, n_jobs = -1)
rnd_clf.fit(X_train,y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [ ]:
#BaggingClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features = 'auto', max_leaf_nodes = 16),
    n_estimators = 500, max_samples = 1.0, bootstrap = True, n_jobs = -1
)

### 1) 엑스트라 트리

: 극단적으로 무작위한 트리의 랜덤 포레스트/ 편향이 늘어나지만 대신 분산을 낮춤

### 2) 특성 중요도

* 사이킷런은 훈련이 끝난 뒤 특성마다 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결괏값을 정규화함

In [9]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)
rnd_clf.fit(iris['data'],iris['target'])
for name, score in zip(iris['feature_names'], rnd_clf.feature_importances_) :
  print(name,score)

sepal length (cm) 0.09691361057420006
sepal width (cm) 0.02345681133972607
petal length (cm) 0.4275133676147639
petal width (cm) 0.45211621047131006


---
## 7.5 부스팅

: **약한 학습기**를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법

### 1) 에이다부스트

* 이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 **과소적합**했던 훈련 샘플의 가중치를 더 높이는 것
* 에이다부스트 앙상블이 훈련 세트에 **과대적합**되면 추정기 수를 줄이거나 추정기의 규제를 더 강화하기

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DeicisonTreeClassifier(max_depth = 1), n_estimators = 200,
    algorithm = 'SAMME.R', learning_rate = 0.5
)

ada_clf.fit(X_train,y_train)

### 2) 그레이디언트 부스팅

* 에이다부스트처럼 앙상블에 이전의 오차를 보정하도록 예측기를 순차적으로 추가하지만 반복마다 샘플의 가중치를 수정하는 대신 이전 에측기가 만든 **잔여 오차**에 새로운 예측기를 학습시킴

* `learning_rate` 매개변수가 각 트리의 기여 정도를 조절함

In [10]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth = 2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [11]:
#첫 번째 예측기에서 생긴 잔여 오차에 두번쨰 DT 훈련

y2 = y-tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth = 2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [12]:
#두 번째 예측기가 만든 잔여 오차에 세 번쨰 회귀모델 훈련

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth = 2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [14]:
#새로운 샘플에 대한 예측을 만들기 위해 모든 트리의 예측 더하면 됨

y_pred = sum(tree.predict(X) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [16]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 3, learning_rate = 1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

* 최적의 트리 수를 찾기 위해서는 조기 종료 기법을 사용할 수 있음 => `staged_predict()`

* 훈련의 각 단계에서 앙상블에 의해 만들어진 예측기를 순회하는 반복자를 반환

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train,X_val,y_train,y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth = 2, n_estimators = 120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val,y_pred)
for y_pred in gbrt.staged_predict(X_val)]

bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth = 2, n_estimators = bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=117)

* 실제로 훈련을 중지하는 방법으로 조기 종료 구현

`warm_start = True`로 설정하면 사이킷런이 fit() 메서드가 호출될 때 기존 트리를 유지하고 훈련을 추가할 수 있도록 해줌

* 연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춤

In [19]:
gbrt = GradientBoostingRegressor(max_depth = 2, warm_start = True)

min_val_error = float('inf')
error_going_up = 0

for n_estimators in range(1,120) :
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train,y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val, y_pred)
  if val_error < min_val_error :
    min_val_error = val_error
    error_going_up = 0
  else :
    error_going_up += 1
    if error_going_up == 5 :
      break

* 각 트리가 훈련 될 때 사용할 훈련 샘플의 비율 지정 `subsample = 0.25`
=> **확률적 그레이디언트 부스팅**


#### (1) XGB

In [21]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [22]:
xgb_reg.fit(X_train,y_train,
            eval_set = [(X_val, y_val)], early_stopping_rounds = 2)
y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:0.40507
[1]	validation_0-rmse:0.35347
[2]	validation_0-rmse:0.32152
[3]	validation_0-rmse:0.30545
[4]	validation_0-rmse:0.29699
[5]	validation_0-rmse:0.29607
[6]	validation_0-rmse:0.30157
[7]	validation_0-rmse:0.29921


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


---
## 7.6 스태킹

* 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는  모델을 훈련시킬 수 없나?!

1. 훈련 세트를 두 개의 서브셋으로 나눔. 첫 번째 서브셋은 첫번재 레이어의 예측을 훈련시키기 위해 사용됨
2. 첫 번째 레이어의 예측기를 사용해 두 번쨰 세트에 대한 예측을 만듦. 예측은 완전히 새로운 것
3. 홀드 아웃 세트(두번쨰)의 각 샘플에 대해 세 개의 예측값이 있음. 타깃값은 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트를 만들 수 있음(3차원)
4. 첫 번째 레이어의 예측을 가지고 타깃값을 예측하도록 학습